# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [19]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import openai

In [20]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
api_endpoint = os.getenv('OPENAI_ENDPOINT')
api_deploymentName = os.getenv('OPENAI_DEPLOYMENTNAME')

if api_key:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = api_deploymentName

openai = openai.AzureOpenAI(
    api_key = api_key,
    azure_endpoint = api_endpoint,
    api_version="2024-02-15-preview"
)

API key looks good so far


In [21]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a') if link.get('href') and not link.get('href').startswith('mailto:')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [22]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [23]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [24]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [25]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [26]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [27]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [28]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen2.5-Omni-7B',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/all-hands/openhands-lm-32b-v0.1',
 '/ByteDance/MegaTTS3',
 '/openfree/flux-chatgpt-ghibli-lora',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/Anthropic/EconomicIndex',
 '/datasets/a-m-team/AM-DeepSeek-R1-Distilled-1.4M',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '

In [35]:
urls = [
    "https://huggingface.co",
    "https://woffu.com/es/",
    #"https://neoris.com/es/web/guest/home"
]

for url in urls:
    links = get_links(url)
    print(f"Links encontrados en {url}:")
    print(links)
    print("-" * 50)



Links encontrados en https://huggingface.co:
{'links': [{'url': 'https://huggingface.co/about', 'type': 'about page'}, {'url': 'https://huggingface.co/careers', 'type': 'careers page'}]}
--------------------------------------------------
Links encontrados en https://woffu.com/es/:
{'links': [{'url': 'https://woffu.com/es/quienes-somos/'}, {'url': 'https://woffu.com/es/sostenibilidad/'}, {'url': 'https://jobs.woffu.com/'}]}
--------------------------------------------------


## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [65]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    #print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [37]:
#print(get_all_details("https://huggingface.co"))

for url in urls:
    links = get_all_details(url)
    print(f"Links encontrados en {url}:")
    print(links)
    print("-" * 50)

Found links: {'links': [{'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}]}
Links encontrados en https://huggingface.co:
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen2.5-Omni-7B
Updated
3 days ago
•
70.7k
•
1.11k
deepseek-ai/DeepSeek-V3-0324
Updated
8 days ago
•
143k
•
2.27k
all-hands/openhand

In [38]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [51]:
def get_brochure_user_prompt(company_name, url, language="en"):
    prompts = {
        "en": (
            f"You are looking at a company called: {company_name}\n"
            f"Here are the contents of its landing page and other relevant pages; "
            f"use this information to build a short brochure of the company in markdown.\n"
        ),
        "es": (
            f"Estás viendo una empresa llamada: {company_name}\n"
            f"Aquí están los contenidos de su página de inicio y otras páginas relevantes; "
            f"usa esta información para crear un folleto corto de la empresa en markdown.\n"
        )
    }
    
    user_prompt = prompts.get(language, prompts["en"])  # Predeterminado a inglés si el idioma no es válido
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]  # Truncar si supera los 5,000 caracteres
    
    return user_prompt

In [52]:
get_brochure_user_prompt("Woffu Job Organizer", "https://woffu.com/es/", "es")

Found links: {'links': [{'type': 'contact page', 'url': 'https://woffu.com/es/contacto/'}, {'type': 'control de horario', 'url': 'https://woffu.com/es/control-horario-y-registro-de-la-jornada-laboral/'}, {'type': 'control de vacaciones', 'url': 'https://woffu.com/es/control-de-vacaciones-y-ausencias-de-empleados/'}, {'type': 'about page', 'url': 'https://woffu.com/es/quienes-somos/'}, {'type': 'sustainability page', 'url': 'https://woffu.com/es/sostenibilidad/'}, {'type': 'clients page', 'url': 'https://woffu.com/es/clientes-woffu/'}, {'type': 'jobs/careers page', 'url': 'https://jobs.woffu.com/'}]}


'Estás viendo una empresa llamada: Woffu Job Organizer\nAquí están los contenidos de su página de inicio y otras páginas relevantes; usa esta información para crear un folleto corto de la empresa en markdown.\nLanding page:\nWebpage Title:\nGestión del tiempo adaptado a la legalidad con Woffu\nWebpage Contents:\nContacto\nIniciar sesión\nProducto\nControl horario\nGestiona la política horaria y mejora el bienestar laboral.\nGestor de vacaciones y ausencias\nOptimiza el proceso de solicitudes y gana visibilidad.\nApp control horario y fichaje\nFicha y gestiona las ausencias desde un dispositivo móvil.\nInformes RRHH\nToma decisiones estratégicas basadas en datos reales.\nGestión de turnos\nAlcanza una planificación a medida y ordenada.\nComunicación interna\nFacilita la comunicación dentro de tu empresa.\nCanal de Denuncias\nCumple con la normativa Whistleblowing.\nPor qué Woffu\nEspecialistas en tu sector\nAlimentación\nFarmacéutico\nAutomoción\nMedia\nCumple la normativa laboral\nAseg

In [60]:
def create_brochure(company_name, url, language="en"):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, language)}
        ],
    )
    result = response.choices[0].message.content

    if language == "es":
        translation = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "Traduce el siguiente texto al español:"},
                {"role": "user", "content": result}
            ],
        )
        result = translation.choices[0].message.content

    display(Markdown(result))


In [61]:
create_brochure("Woffu Job Organizer", "https://woffu.com/es/", "es")

Found links: {'links': [{'url': 'https://woffu.com/es/', 'type': 'company page'}, {'url': 'https://woffu.com/es/quienes-somos/', 'type': 'about page'}, {'url': 'https://jobs.woffu.com/', 'type': 'careers page'}]}


# Organizador de Trabajo Woffu - Optimiza tu Gestión del Tiempo

¡Bienvenidos a Woffu Job Organizer! Somos líderes en la digitalización de la gestión del tiempo, adaptándonos a la legalidad y resolviendo situaciones complejas de Recursos Humanos. Más de 1,500 clientes confían en nosotros para mejorar la eficiencia y el bienestar laboral de sus equipos.

## Nuestra Solución

- **Control Horario**: Gestiona la política horaria y mejora el bienestar laboral de tu empresa.
- **Gestor de Vacaciones y Ausencias**: Optimiza el proceso de solicitudes y obtén visibilidad sobre el tiempo de tus empleados.
- **App Control Horario y Fichaje**: Registra y gestiona las ausencias desde cualquier dispositivo móvil.
- **Informes de RRHH**: Toma decisiones estratégicas basadas en datos reales y actualizados.
- **Gestión de Turnos**: Logra una planificación personalizada y ordenada para tu equipo.
- **Comunicación Interna**: Facilita la comunicación dentro de tu empresa para una mejor coordinación.
- **Canal de Denuncias**: Cumple con la normativa Whistleblowing y fomenta un entorno seguro y transparente.

## Por Qué Elegirnos

- **Especialistas en Tu Sector**: Atendemos a diversos sectores como alimentación, farmacéutico, automoción y medios de comunicación.
- **Cumplimos con la Normativa Laboral**: Garantizamos el registro digital obligatorio y la reducción de la jornada laboral.
- **Servicio Postventa de Calidad**: Acompañamos a nuestros clientes para garantizar su satisfacción y éxito.
- **Implementación Guiada**: Te acompañamos en una puesta en marcha rápida y sencilla.
- **Integraciones**: Conecta nuestra solución con tus sistemas para potenciar su uso y eficacia.

## Experiencias de Clientes

"Con Woffu, nos ha sido fácil mantener actualizada la gestión del tiempo de nuestros empleados, ganando visibilidad y tranquilidad en el proceso."
- Carlos Alonso, Director de Recursos Humanos en Eurofred

"La agilidad y automatización que nos brinda Woffu ha revolucionado la gestión de RRHH en nuestra institución educativa."
- María Villegas, Técnico de RRHH en UNIR

## ¡Únete a Nosotros!

Si buscas optimizar tu gestión del tiempo, reducir costos y mejorar la experiencia de tus empleados, ¡Woffu es tu solución! Solicita una demo gratuita y descubre cómo podemos ayudarte a alcanzar tus objetivos.

¡Contáctanos hoy y comienza a transformar la gestión del tiempo en tu empresa con Woffu Job Organizer!

---
©2025 Woffu Job Organizer, SL. Todos los derechos reservados.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [69]:
def stream_brochure(company_name, url, language="en"):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, language)}
        ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta and hasattr(chunk.choices[0].delta, "content"):
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```", "").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)


In [70]:
stream_brochure("Woffu Job Organizer", "https://woffu.com/es/", "es")

# Woffu Job Organizer - Optimización en la gestión del tiempo laboral

¡Bienvenido a Woffu Job Organizer! Somos líderes en la digitalización de la gestión del tiempo, ofreciendo soluciones para simplificar y mejorar la organización laboral de tu empresa. Conócenos más a fondo:

## Sobre Nosotros
En Woffu, nos comprometemos a ofrecerte una gestión del tiempo adaptada a la legalidad laboral. Nuestro enfoque se centra en resolver las casuísticas complejas de Recursos Humanos, permitiéndote optimizar recursos y mejorar el bienestar laboral.

## Nuestros Clientes
Más de 3,000 empresas confían en nosotros, provenientes de diversos sectores como alimentación, farmacéutico, automoción y medios de comunicación. Nuestro compromiso es garantizar el cumplimiento de la normativa laboral y brindar un servicio postventa de calidad para tu satisfacción.

## Cultura Corporativa
En Woffu, promovemos la confianza y autonomía en nuestros clientes, permitiéndoles gestionar su tiempo de forma eficiente y transparente. Valoramos la comunicación interna, la sostenibilidad y el compromiso con un futuro más verde.

## Carreras y Empleo
¿Buscas formar parte de un equipo apasionado y experimentado? En Woffu, valoramos la excelencia en el servicio al cliente y la innovación en la gestión del tiempo. Explora oportunidades laborales y únete a nosotros para contribuir a la transformación digital de los procesos de Recursos Humanos.

Si deseas optimizar la gestión del tiempo en tu empresa y obtener una solución integral y especializada, ¡contacta con nosotros para solicitar una demo gratuita!

Solicita una Demo  
Hazte partner  
Help Center  

¡En Woffu Job Organizer, simplificamos la gestión del tiempo para potenciar el éxito de tu empresa! 🌟

In [71]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co", "es")

# Hugging Face - Building the Future

Welcome to Hugging Face, the AI community revolutionizing the future of technology. Our platform serves as a collaborative space where the machine learning community comes together to work on models, datasets, and applications.

## Company Culture

At Hugging Face, we pride ourselves on fostering a culture of innovation, collaboration, and creativity. We believe in the power of community-driven development, where individuals can share their expertise, learn from one another, and push the boundaries of AI technology.

## Customers

Join over 50,000 organizations, including industry giants like Amazon, Google, and Microsoft, in leveraging Hugging Face's cutting-edge solutions. We provide Enterprise options with advanced security features, access controls, and dedicated support, tailored to meet the needs of your team.

## Careers and Jobs

Looking to join our dynamic team? Explore exciting career opportunities at Hugging Face. Be part of a community passionate about pushing AI boundaries and creating impactful solutions.

Visit our [website](https://huggingface.co) for more information on our models, datasets, spaces, and opportunities to collaborate. Connect with us on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/hugging-face/), and [Discord](https://discord.com/invite/4W9VFP2) to stay updated on the latest developments.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>